In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense,Embedding,RepeatVector
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
from numpy import array,random,take

import string
import re
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential

In [ ]:
from numpy import argmax

In [ ]:
data_path='/content/sample_data/fra-eng/fra.txt'
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read()

In [ ]:
lines


In [ ]:
def to_lines(text):
  sents=text.strip().split("\n")
  sents=[i.split('\t') for i in sents]
  return sents

In [ ]:
fra_eng=to_lines(lines)
fra_eng[:5]

[['Go.',
  'Va !',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)'],
 ['Go.',
  'Marche.',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)'],
 ['Go.',
  'En route !',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)'],
 ['Go.',
  'Bouge !',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)'],
 ['Hi.',
  'Salut !',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)']]

In [ ]:
fra_eng=array(fra_eng)
fra_eng[:5]

array([['Go.', 'Va !',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)'],
       ['Go.', 'Marche.',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)'],
       ['Go.', 'En route !',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)'],
       ['Go.', 'Bouge !',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)'],
       ['Hi.', 'Salut !',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)']],
      dtype='<U349')

In [ ]:
fra_eng.shape

(229803, 3)

In [ ]:
fra_eng=fra_eng[:50000,:]

In [ ]:
fra_eng=fra_eng[:,[0,1]]
fra_eng[:5]

array([['Go.', 'Va !'],
       ['Go.', 'Marche.'],
       ['Go.', 'En route !'],
       ['Go.', 'Bouge !'],
       ['Hi.', 'Salut !']], dtype='<U349')

DATA CLEANING

In [ ]:
#REMOVE PUNCTUATION
fra_eng[:,0]=[s.translate(str.maketrans('','',string.punctuation))for s in fra_eng[:,0] ]
fra_eng[:,1]=[s.translate(str.maketrans('','',string.punctuation))for s in fra_eng[:,1] ]

In [ ]:
fra_eng[:5]

array([['Go', 'Va '],
       ['Go', 'Marche'],
       ['Go', 'En route '],
       ['Go', 'Bouge '],
       ['Hi', 'Salut ']], dtype='<U349')

In [ ]:
#convert text to lower case
for i in range(len(fra_eng)):
  fra_eng[i,0]=fra_eng[i,0].lower()
  fra_eng[i,1]=fra_eng[i,1].lower()
fra_eng

array([['go', 'va '],
       ['go', 'marche'],
       ['go', 'en route '],
       ...,
       ['im right behind tom', 'je suis juste derrière tom'],
       ['im right behind you', 'je suis juste derrière toi'],
       ['im right behind you', 'je suis juste derrière vous']],
      dtype='<U349')

text to sequence conversion(word to index mapping)
1.convert sentences into numbers
2.every sentence should be of same length

In [ ]:
#function to build a tokenizer
def tokenization(lines):
  tokenizer=Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer
#prepare english tokenizer
eng_tokenizer=tokenization(fra_eng[:,0])
eng_vocab_size=len(eng_tokenizer.word_index)+1
eng_length=8
print(eng_vocab_size)

5954


In [ ]:
#prepare english tokenizer
fra_tokenizer=tokenization(fra_eng[:,1])
fra_vocab_size=len(fra_tokenizer.word_index)+1
fra_length=8
print(fra_vocab_size)


13706


In [ ]:
#encode and pad sequences
def encode_sequences(tokenizer,length,lines):
  #integer encode sequences
  seq=tokenizer.texts_to_sequences(lines)
  #pad sequences with 0
  seq=pad_sequences(seq,maxlen=length,padding='post')
  return seq


In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(fra_eng,test_size=0.2,random_state=3)


In [ ]:
#prepare training data
trainX=encode_sequences(fra_tokenizer,fra_length,train[:,1])
trainY=encode_sequences(eng_tokenizer,eng_length,train[:,0])
#prepare validation data
testX=encode_sequences(fra_tokenizer,fra_length,test[:,1])
testY=encode_sequences(eng_tokenizer,eng_length,test[:,0])

define seq2seq2 model

In [ ]:
def define_model(in_vocab,out_vocab,in_timesteps,out_timesteps,units):
  model=Sequential()
  model.add(Embedding(in_vocab,units,input_length=in_timesteps,mask_zero='True'))
  model.add(LSTM(units))
  model.add(RepeatVector(out_timesteps))
  model.add(LSTM(units,return_sequences='True'))
  model.add(Dense(out_vocab,activation="softmax"))
  return model


In [ ]:
model=define_model(fra_vocab_size,eng_vocab_size,fra_length,eng_length,512)
rms=optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=rms,loss='sparse_categorical_crossentropy')

In [ ]:
#train the model
model.fit(trainX,trainY.reshape(trainY.shape[0],trainY.shape[1],1),
          batch_size=512,
          epochs=50,
          validation_split=0.2)

Epoch 1/50
63/63 [==============================] - 13s 116ms/step - loss: 4.2239 - val_loss: 3.1206
Epoch 2/50
63/63 [==============================] - 4s 68ms/step - loss: 2.8938 - val_loss: 2.7815
Epoch 3/50
63/63 [==============================] - 4s 70ms/step - loss: 2.7442 - val_loss: 2.7589
Epoch 4/50
63/63 [==============================] - 4s 71ms/step - loss: 2.6972 - val_loss: 2.6929
Epoch 5/50
63/63 [==============================] - 4s 61ms/step - loss: 2.6672 - val_loss: 2.6707
Epoch 6/50
63/63 [==============================] - 4s 61ms/step - loss: 2.6493 - val_loss: 2.6671
Epoch 7/50
63/63 [==============================] - 4s 63ms/step - loss: 2.6277 - val_loss: 2.6418
Epoch 8/50
63/63 [==============================] - 4s 62ms/step - loss: 2.6119 - val_loss: 2.6326
Epoch 9/50
63/63 [==============================] - 4s 62ms/step - loss: 2.5975 - val_loss: 2.6313
Epoch 10/50
63/63 [==============================] - 4s 66ms/step - loss: 2.5768 - val_loss: 2.5874
Epoch 1

In [ ]:
pred=model.predict(testX.reshape(testX.shape[0],testX.shape[1],1))

313/313 [==============================] - 3s 4ms/step


In [ ]:
#Get the predicted classes from the predicted probabilities
predicted_classes = pred.argmax(axis=1)

In [ ]:
predicted_classes

array([[7, 0, 3, ..., 3, 1, 1],
       [7, 0, 2, ..., 2, 2, 2],
       [7, 1, 2, ..., 1, 1, 1],
       ...,
       [7, 2, 4, ..., 3, 1, 1],
       [7, 1, 2, ..., 2, 1, 1],
       [7, 0, 3, ..., 3, 2, 2]])

In [ ]:
def get_words(n,tokenizer):
  for word,index in tokenizer.word_index.items():
    if index==n:
      return word
    return None


In [ ]:
pred_text=[]
for i in predicted_classes:
  temp=[]
  for j in range(len(i)):
    t=get_words(i[j],eng_tokenizer)
    if j>0:
      if((t==get_words(i[j-1],eng_tokenizer)) or (t==None)):
        temp.append('')
      else:
        temp.append(t)
    else:
      if(t==None):
        temp.append('')
      else:
        temp.append(t)
  pred_text.append(' '.join(temp))


In [ ]:
pred_text[5]

'     i        i    i       i                 i   i          i  i  i  i       i    i    i  i  i     i    i    i   i      i  i  i    i  i   i        i   i      i    i  i    i  i      i    i      i      i        i    i         i         i    i   i    i  i  i   i   i    i    i   i   i   i       i      i   i  i    i  i  i  i      i  i   i  i  i     i     i     i  i   i  i   i  i   i     i     i       i    i      i  i  i   i          i   i   i   i     i   i    i     i   i     i    i      i      i       i  i    i   i  i         i   i      i  i  i   i  i    i  i      i  i  i  i     i     i      i   i    i       i  i     i   i  i  i      i   i      i   i        i   i  i        i   i    i       i   i  i   i  i   i    i   i      i       i  i        i     i   i      i      i  i   i  i   i         i        i       i           i       i    i    i           i    i   i    i  i  i       i  i     i   i      i  i     i  i    i    i  i    i  i     i        i  i  i        i   i   i       i      i  i  i   

In [ ]:
import pandas as pd
df=pd.DataFrame({"actual": test[:,0], "predicted": pred_text})

In [ ]:
df.sample(15)

,actual,predicted
242,im not a liar,i i i ...
9669,do you like cats,i i i i ...
2430,howd you find it,i i i i i i i i i i ...
3571,the storm relented,i i i i i i ...
5464,its begun,i i i i i i i i i ...
2029,tom is numb,i i i i i i ...
1814,tom is smiling,i i i i i i i i i ...
9660,i stepped backwards,i i i i ...
958,he never tells a lie,i i i i i i i ...
8085,stop the car here,i i i i i i i i i ...
